## Mounting of Google Drive for use in  Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## IMPORTATION OF REQUIRED LIBRARIES

In [ ]:
!pip install torch -q
import pandas as pd
import plotly.express as px
import random
import os
import matplotlib.pyplot as plt
import numpy as np
import time
from torch import Tensor,deg2rad,sin,asin,cos,sqrt#,min,max#
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
path_list=[os.path.join('/content/drive/MyDrive/networks',path) for path in os.listdir('/content/drive/MyDrive/networks')]

## LOADING MY DATA

## PLOTTING THE LOCATIONS OF THE MAP OF EUROPE

## DEFINING MY  DISTANCE FUNCTION

In [ ]:
# THIS FUNCTION CALCULATES THE HAVERSINE DISTANCE BETWEEN TWO POINTS
def haversine(loc1, loc2):
      R = 3959.87433 # this is in miles.  For Earth radius in kilometers use 6372.8 km
      lat1=loc1[:,0]
      lon1=loc1[:,1]
      lat2=loc2[0]
      lon2=loc2[1]
      dLat = deg2rad(lat2 - lat1)# CONVERTING DEGREES TO RADIANS
      dLon = deg2rad(lon2 - lon1)
      lat1 = deg2rad(lat1)
      lat2 = deg2rad(lat2)
      a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
      c = 2*asin(sqrt(a))
      return R * c # RETURNS THE HAVERSINE DISTANCE

In [ ]:
k,l=1,1
m,n=0.001,0.001
p=0.001
q=0.3
z=0.3

## DEFINING OUR OBJECTIVE FUNCTIONS

In [ ]:
# THE FIRST OBJECTIVE FUNCTION HERE CALCULATES AND RETURNS THE MAXIMUM OF THE MINIMUM NODE TO CONTROLLER LATENCY
def maxn2C_fn(city_locs, cont_locs):
  dists = haversine(city_locs,cont_locs[0])
  min_dists = dists
  for i in range(1,cont_locs.shape[0]):
    dists = haversine(city_locs,cont_locs[i])
    min_dists = torch.min(min_dists, dists)
  return min_dists.max()
# THE SECOND OBJECTIVE FUNCTION HERE CALCULATES AND RETURNS THE MEAN OF THE MINIMUM NODE TO CONTROLLER LATENCY
def mean2C_fn(city_locs, cont_locs):
  dists = haversine(city_locs,cont_locs[0])
  min_dists = dists
  for i in range(1,cont_locs.shape[0]):
    dists = haversine(city_locs,cont_locs[i])
    min_dists = torch.min(min_dists, dists)
    return min_dists.mean()
# THE THIRD OBJECTIVE FUNCTION HERE CALCULATES AND RETURNS MAXIMUM INTERCONTROLLER LATENCY
def maxC2C_fn(cont_locs,amb_num):
  dists = haversine(cont_locs,amb_num[0])
  max_dists = dists
  for i in range(1,amb_num.shape[0]):
    dists = haversine(cont_locs,amb_num[i])
    max_dists = torch.max(max_dists, dists)
  return max_dists.max()*0.1
# THE FOURTH OBJECTIVE FUNCTION HERE CALCULATES AND RETURNS DIVISION OF THE SUM OF INTERCONTROLLER LATENCY BY 5C2
def meanC2C_fn(cont_locs,amb_num):
  dists = haversine(cont_locs,amb_num[0])
  sum_dists = dists.sum()
  for i in range(1,amb_num.shape[0]):
    dists = haversine(cont_locs,amb_num[i]).sum()
    sum_dists = sum_dists+ dists
  return (sum_dists/20)*0.1
def load_balancing(city_locs,cont_locs):
   node_counts=np.zeros(cont_locs.shape[0])
   for i in range(city_locs.shape[0]):
    dists = haversine(cont_locs,city_locs[i])
    node_counts[np.argmin(dists.clone().detach().numpy())]+=1
   return np.ptp(node_counts)
def max_tolerance(city_locs,cont_locs):
  dists = haversine(city_locs,cont_locs[0])
  max_dists = dists
  for i in range(1,cont_locs.shape[0]):
    dists = haversine(city_locs,cont_locs[i])
    min_dists = torch.max(max_dists, dists)
  return max_dists.max()
def all_losses(city_locs,cont_locs):
  l1=maxn2C_fn(city_locs,cont_locs).item()
  l2=mean2C_fn(city_locs,cont_locs).item()
  l3=maxC2C_fn(cont_locs,cont_locs).item()
  l4=meanC2C_fn(cont_locs,cont_locs).item()
  l5=max_tolerance(city_locs,cont_locs).item()
  l6=load_balancing(city_locs,cont_locs)
  return np.array([l1,l2,l3,l4,l5,l6])

In [ ]:
def overall_loss(city_locs,cont_locs):
  l1=maxn2C_fn(city_locs,cont_locs)
  l2=mean2C_fn(city_locs,cont_locs)
  l3=maxC2C_fn(cont_locs,cont_locs+0.0001)
  l4=meanC2C_fn(cont_locs,cont_locs+0.0001)
  l5=max_tolerance(city_locs,cont_locs)
  return k*l1 + l*l2+m*l3+n*l4+p*l5
def overall_metric( city_locs,cont_locs):
  l1=maxn2C_fn(city_locs,cont_locs).item()
  l2=mean2C_fn(city_locs,cont_locs).item()
  l3=maxC2C_fn(cont_locs,cont_locs).item()
  l4=meanC2C_fn(cont_locs,cont_locs).item()
  l5=max_tolerance(city_locs,cont_locs).item()
  l6=load_balancing(city_locs,cont_locs)
  return (k*l1 + l*l2+m*l3+n*l4+p*l5)*(l6**q)
def overall_cost(best_loss,no_of_cont):
  return best_loss*(no_of_cont**z)

## GENERATING OUR RANDOM INITIAL LOCATIONS

In [ ]:
import random,numpy
from tqdm import tqdm
from torch import manual_seed

In [ ]:
cost_patience=3
def find_optimal(pos_tensors):
  lr=0.01
  n_epochs = 200
  early_stopping=5
  main_seed=12
  overalls=[]
  opt_no_cont=3
  iter=0
  # dataloader to load our data in batches, here we are using a batch size of 1
  for siz in range(3,min(pos_tensors.shape[0],20)):
    iter+=1
    manual_seed(42)
    # using torch to generate a random of size 5 by 2
    np.random.seed(42)
    rand_cities =np.random.choice(range(pos_tensors.shape[0]),siz,replace=False).astype(int)
    rand_cities=np.sort(rand_cities)
    cont_locs=torch.tensor(pos_tensors.numpy()[rand_cities.astype(int)])# adding the mean of our longitudes and latitudes to enable faster convergence of the model
    initial_locs = cont_locs.clone() # Save originals for later
    cont_locs=cont_locs+0.0001 # adding little noise for calculation of gradients
    cont_locs.requires_grad_(True)
    val_losses = []
    losses_sum=[]
    verbose=4
    final_locs=cont_locs.clone()
    for epoch in range(n_epochs):
    # Run through batches
      loss =overall_loss(pos_tensors,cont_locs)
      loss.backward() # Calc grads
      cont_locs.data -= lr * cont_locs.grad.data # Update locs
      cont_locs.grad = None # Reset gradients for next step
      losses=overall_metric(pos_tensors,cont_locs)
      if epoch>0 :
        if losses< np.min(losses_sum):
          final_locs=cont_locs.clone()
          best_loss=val_loss
        else :
          best_loss=np.min(losses_sum)
      losses_sum.append(losses)
      val_loss=all_losses(pos_tensors,cont_locs)
      #if epoch % verbose==0:
         #print("After {0:.0f} epochs: maxN2c: {1:.2f} meanN2C: {2:.2f} maxICL: {3:.2f} meanICL: {4:.2f} Max Tolerance:{5:.2f} Load Balancing:{6:.2f} Overall: {7:.2f} ".format(epoch,val_loss[0],val_loss[1],val_loss[2],val_loss[3],val_loss[4],val_loss[5],losses))
      if epoch > early_stopping:
        if np.min(losses_sum[-early_stopping:])> np.min(losses_sum):
        #  print ('Training ended by early stopping, best score at epoch {}'.format(epoch))
          break
    #print("Best scores at epoch {0} maxN2c: {1:.2f} meanN2C: {2:.2f} maxICL: {3:.2f} meanICL: {4:.2f} Overall: {5:.2f} ".format(epoch,best_loss[0],best_loss[1],best_loss[2],best_loss[3],best_loss.sum()))
    if iter>1:
      if overall_cost(best_loss,siz)<np.min(overalls):
        opt_no_cont= siz
    overalls.append(overall_cost(best_loss,siz))
  return opt_no_cont

In [ ]:
fields=['lon','lat']
lats=[]
longs=[]
opts=[]
for path in tqdm(path_list[:-1]):
  pos_data=pd.read_csv(path,usecols=fields) #reading from the raw Csv
  pos_tensors=torch.Tensor(pos_data[['lat','lon']].values) #converting the longitudes and the latitudes to tensor format
  lats.append(pos_data['lat'].values.tolist())
  longs.append(pos_data['lon'].values.tolist())
  opts.append(find_optimal(pos_tensors))

100%|██████████| 3/3 [01:02<00:00, 20.86s/it]


In [ ]:
df=pd.DataFrame({'lats':lats,'longs':longs})
df['no_of_cont']=opts
df.to_csv('cont_opt.csv',index=False)